In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from xgboost.sklearn import XGBRegressor, XGBClassifier

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337


In [5]:
train_x = train.drop(['ID', '수요량'], axis=1)
train_y = train['수요량']

test_x = test.drop(['ID'], axis=1)

In [6]:
ordinal_features = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])
    
    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[feature] = le.transform(test_x[feature])

In [11]:
train_x.head()

,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,1,6,212000,0,20,0,1,9
1,2,69,113000,0,13,0,0,15
2,0,6,67000,0,20,0,1,22
3,1,4,206000,0,3,2,0,32
4,1,61,140000,0,7,1,1,30


In [9]:
model = DecisionTreeRegressor(random_state=42)

In [10]:
model.fit(train_x, train_y)

DecisionTreeRegressor(random_state=42)

In [12]:
preds = model.predict(test_x)

In [14]:
preds

array([  55.,   26.,  378., ...,  189., 1026.,  298.])

In [15]:
submission = pd.read_csv('./data/sample_submission.csv')

In [17]:
submission['수요량'] = preds
submission.head()

,ID,수요량
0,TEST_0000,55.0
1,TEST_0001,26.0
2,TEST_0002,378.0
3,TEST_0003,109.0
4,TEST_0004,271.0


In [19]:
submission.to_csv('./output/DecisionTreeRegressor.csv', index=False)